### Problem Statement_[Data Science for Good: Kiva Crowdfunding](https://www.kaggle.com/kiva/data-science-for-good-kiva-crowdfunding)
Kiva.org is an online crowdfunding platform to extend financial services to poor and financially excluded people around the world. For the locations in which Kiva has active loans, your objective is to pair Kiva's data with additional data sources to estimate the welfare level of borrowers in specific regions, based on shared economic and demographic characteristics.

A good solution would connect the features of each loan or product to one of several poverty mapping datasets, which indicate the average level of welfare in a region on as granular a level as possible. Many datasets indicate the poverty rate in a given area, with varying levels of granularity. Kiva would like to be able to disaggregate these regional averages by gender, sector, or borrowing behavior in order to estimate a Kiva borrower’s level of welfare using all of the relevant information about them. Strong submissions will attempt to map vaguely described locations to more accurate geocodes.

Kernels submitted will be evaluated based on the following criteria:

1. Localization - How well does a submission account for highly localized borrower situations? Leveraging a variety of external datasets and successfully building them into a single submission will be crucial.

2. Execution - Submissions should be efficiently built and clearly explained so that Kiva’s team can readily employ them in their impact calculations.

3. Ingenuity - While there are many best practices to learn from in the field, there is no one way of using data to assess welfare levels. It’s a challenging, nuanced field and participants should experiment with new methods and diverse datasets.

In [1]:
import pandas as pd

### Original Data From Kaggle

In [2]:
loans = pd.read_csv('/Users/lasayin/Desktop/Kiva_Crowdfunding/OriginalData/kiva_loans.csv')

In [3]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671205 entries, 0 to 671204
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  671205 non-null  int64  
 1   funded_amount       671205 non-null  float64
 2   loan_amount         671205 non-null  float64
 3   activity            671205 non-null  object 
 4   sector              671205 non-null  object 
 5   use                 666973 non-null  object 
 6   country_code        671197 non-null  object 
 7   country             671205 non-null  object 
 8   region              614405 non-null  object 
 9   currency            671205 non-null  object 
 10  partner_id          657698 non-null  float64
 11  posted_time         671205 non-null  object 
 12  disbursed_time      668809 non-null  object 
 13  funded_time         622874 non-null  object 
 14  term_in_months      671205 non-null  float64
 15  lender_count        671205 non-nul

In [4]:
loans.head()

,id,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,date
0,653051,300.0,300.0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:12:39+00:00,2013-12-17 08:00:00+00:00,2014-01-02 10:06:32+00:00,12.0,12,NaN,female,irregular,2014-01-01
1,653053,575.0,575.0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 06:51:08+00:00,2013-12-17 08:00:00+00:00,2014-01-02 09:17:23+00:00,11.0,14,NaN,"female, female",irregular,2014-01-01
2,653068,150.0,150.0,Transportation,Transportation,To repair their old cycle-van and buy another ...,IN,India,Maynaguri,INR,334.0,2014-01-01 09:58:07+00:00,2013-12-17 08:00:00+00:00,2014-01-01 16:01:36+00:00,43.0,6,"user_favorite, user_favorite",female,bullet,2014-01-01
3,653063,200.0,200.0,Embroidery,Arts,to purchase an embroidery machine and a variet...,PK,Pakistan,Lahore,PKR,247.0,2014-01-01 08:03:11+00:00,2013-12-24 08:00:00+00:00,2014-01-01 13:00:00+00:00,11.0,8,NaN,female,irregular,2014-01-01
4,653084,400.0,400.0,Milk Sales,Food,to purchase one buffalo.,PK,Pakistan,Abdul Hakeem,PKR,245.0,2014-01-01 11:53:19+00:00,2013-12-17 08:00:00+00:00,2014-01-01 19:18:51+00:00,14.0,16,NaN,female,monthly,2014-01-01


In [5]:
locations = pd.read_csv('/Users/lasayin/Desktop/Kiva_Crowdfunding/OriginalData/kiva_mpi_region_locations.csv')

In [6]:
locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2772 entries, 0 to 2771
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   LocationName  984 non-null    object 
 1   ISO           1008 non-null   object 
 2   country       1008 non-null   object 
 3   region        984 non-null    object 
 4   world_region  1008 non-null   object 
 5   MPI           984 non-null    float64
 6   geo           2772 non-null   object 
 7   lat           892 non-null    float64
 8   lon           892 non-null    float64
dtypes: float64(3), object(6)
memory usage: 195.0+ KB


In [7]:
locations.head()

,LocationName,ISO,country,region,world_region,MPI,geo,lat,lon
0,"Badakhshan, Afghanistan",AFG,Afghanistan,Badakhshan,South Asia,0.387,"(36.7347725, 70.81199529999999)",36.734772,70.811995
1,"Badghis, Afghanistan",AFG,Afghanistan,Badghis,South Asia,0.466,"(35.1671339, 63.7695384)",35.167134,63.769538
2,"Baghlan, Afghanistan",AFG,Afghanistan,Baghlan,South Asia,0.300,"(35.8042947, 69.2877535)",35.804295,69.287754
3,"Balkh, Afghanistan",AFG,Afghanistan,Balkh,South Asia,0.301,"(36.7550603, 66.8975372)",36.755060,66.897537
4,"Bamyan, Afghanistan",AFG,Afghanistan,Bamyan,South Asia,0.325,"(34.8100067, 67.8212104)",34.810007,67.821210


In [8]:
loan_theme_ids = pd.read_csv('/Users/lasayin/Desktop/Kiva_Crowdfunding/OriginalData/loan_theme_ids.csv')

In [9]:
loan_theme_ids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779092 entries, 0 to 779091
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               779092 non-null  int64  
 1   Loan Theme ID    764279 non-null  object 
 2   Loan Theme Type  764279 non-null  object 
 3   Partner ID       764279 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 23.8+ MB


In [10]:
loan_theme_ids.head()

,id,Loan Theme ID,Loan Theme Type,Partner ID
0,638631,a1050000000skGl,General,151.0
1,640322,a1050000000skGl,General,151.0
2,641006,a1050000002X1ij,Higher Education,160.0
3,641019,a1050000002X1ij,Higher Education,160.0
4,641594,a1050000002VbsW,Subsistence Agriculture,336.0


In [11]:
loan_themes_by_region = pd.read_csv('/Users/lasayin/Desktop/Kiva_Crowdfunding/OriginalData/loan_themes_by_region.csv')

In [12]:
loan_themes_by_region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15736 entries, 0 to 15735
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Partner ID          15736 non-null  int64  
 1   Field Partner Name  15736 non-null  object 
 2   sector              15736 non-null  object 
 3   Loan Theme ID       15736 non-null  object 
 4   Loan Theme Type     15736 non-null  object 
 5   country             15736 non-null  object 
 6   forkiva             15736 non-null  object 
 7   region              15736 non-null  object 
 8   geocode_old         1200 non-null   object 
 9   ISO                 15722 non-null  object 
 10  number              15736 non-null  int64  
 11  amount              15736 non-null  int64  
 12  LocationName        15736 non-null  object 
 13  geocode             13662 non-null  object 
 14  names               13661 non-null  object 
 15  geo                 15736 non-null  object 
 16  lat 

In [13]:
loan_themes_by_region.head()

,Partner ID,Field Partner Name,sector,Loan Theme ID,Loan Theme Type,country,forkiva,region,geocode_old,ISO,...,amount,LocationName,geocode,names,geo,lat,lon,mpi_region,mpi_geo,rural_pct
0,9,KREDIT Microfinance Institution,General Financial Inclusion,a1050000000slfi,Higher Education,Cambodia,No,Banteay Meanchey,"(13.75, 103.0)",KHM,...,450,"Banteay Meanchey, Cambodia","[(13.6672596, 102.8975098)]",Banteay Meanchey Province; Cambodia,"(13.6672596, 102.8975098)",13.667260,102.897510,"Banteay Mean Chey, Cambodia","(13.6672596, 102.8975098)",90.0
1,9,KREDIT Microfinance Institution,General Financial Inclusion,a10500000068jPe,Vulnerable Populations,Cambodia,No,Battambang Province,NaN,KHM,...,20275,"Battambang Province, Cambodia","[(13.0286971, 102.989615)]",Battambang Province; Cambodia,"(13.0286971, 102.989615)",13.028697,102.989615,"Banteay Mean Chey, Cambodia","(13.6672596, 102.8975098)",90.0
2,9,KREDIT Microfinance Institution,General Financial Inclusion,a1050000000slfi,Higher Education,Cambodia,No,Battambang Province,NaN,KHM,...,9150,"Battambang Province, Cambodia","[(13.0286971, 102.989615)]",Battambang Province; Cambodia,"(13.0286971, 102.989615)",13.028697,102.989615,"Banteay Mean Chey, Cambodia","(13.6672596, 102.8975098)",90.0
3,9,KREDIT Microfinance Institution,General Financial Inclusion,a10500000068jPe,Vulnerable Populations,Cambodia,No,Kampong Cham Province,"(12.0, 105.5)",KHM,...,604950,"Kampong Cham Province, Cambodia","[(12.0982918, 105.3131185)]",Kampong Cham Province; Cambodia,"(12.0982918, 105.3131185)",12.098292,105.313119,"Kampong Cham, Cambodia","(11.9924294, 105.4645408)",90.0
4,9,KREDIT Microfinance Institution,General Financial Inclusion,a1050000002X1Uu,Sanitation,Cambodia,No,Kampong Cham Province,"(12.0, 105.5)",KHM,...,275,"Kampong Cham Province, Cambodia","[(12.0982918, 105.3131185)]",Kampong Cham Province; Cambodia,"(12.0982918, 105.3131185)",12.098292,105.313119,"Kampong Cham, Cambodia","(11.9924294, 105.4645408)",90.0


### Additional Data

In [14]:
MPI_subnational = pd.read_csv('/Users/lasayin/Desktop/Kiva_Crowdfunding/AdditionalData/MPI_national.csv')

In [15]:
MPI_subnational.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ISO                             102 non-null    object 
 1   Country                         102 non-null    object 
 2   MPI Urban                       102 non-null    float64
 3   Headcount Ratio Urban           102 non-null    float64
 4   Intensity of Deprivation Urban  102 non-null    float64
 5   MPI Rural                       102 non-null    float64
 6   Headcount Ratio Rural           102 non-null    float64
 7   Intensity of Deprivation Rural  102 non-null    float64
dtypes: float64(6), object(2)
memory usage: 6.5+ KB


In [16]:
MPI_subnational.head()

,ISO,Country,MPI Urban,Headcount Ratio Urban,Intensity of Deprivation Urban,MPI Rural,Headcount Ratio Rural,Intensity of Deprivation Rural
0,KAZ,Kazakhstan,0.000,0.0,33.3,0.000,0.09,33.3
1,SRB,Serbia,0.000,0.1,41.4,0.002,0.50,40.3
2,KGZ,Kyrgyzstan,0.000,0.1,40.2,0.003,0.70,37.1
3,TUN,Tunisia,0.000,0.1,35.6,0.012,3.18,38.7
4,ARM,Armenia,0.001,0.2,33.3,0.001,0.39,36.9


In [17]:
MPI_subnational = pd.read_csv('/Users/lasayin/Desktop/Kiva_Crowdfunding/AdditionalData/MPI_subnational.csv')

In [18]:
MPI_subnational.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 8 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ISO country code                   984 non-null    object 
 1   Country                            984 non-null    object 
 2   Sub-national region                984 non-null    object 
 3   World region                       984 non-null    object 
 4   MPI National                       984 non-null    float64
 5   MPI Regional                       984 non-null    float64
 6   Headcount Ratio Regional           984 non-null    float64
 7   Intensity of deprivation Regional  983 non-null    float64
dtypes: float64(4), object(4)
memory usage: 61.6+ KB


In [19]:
MPI_subnational.head()

,ISO country code,Country,Sub-national region,World region,MPI National,MPI Regional,Headcount Ratio Regional,Intensity of deprivation Regional
0,AFG,Afghanistan,Badakhshan,South Asia,0.295,0.387,67.5,57.3
1,AFG,Afghanistan,Badghis,South Asia,0.295,0.466,79.3,58.8
2,AFG,Afghanistan,Baghlan,South Asia,0.295,0.300,59.7,50.3
3,AFG,Afghanistan,Balkh,South Asia,0.295,0.301,55.7,54.1
4,AFG,Afghanistan,Bamyan,South Asia,0.295,0.325,61.0,53.3


In [20]:
World_Population = pd.read_csv('/Users/lasayin/Desktop/Kiva_Crowdfunding/AdditionalData/WorldPopulation.csv')

In [21]:
World_Population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 62 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country         217 non-null    object 
 1   Country Code    217 non-null    object 
 2   Indicator Name  217 non-null    object 
 3   Indicator Code  217 non-null    object 
 4   1960            214 non-null    float64
 5   1961            214 non-null    float64
 6   1962            214 non-null    float64
 7   1963            214 non-null    float64
 8   1964            214 non-null    float64
 9   1965            214 non-null    float64
 10  1966            214 non-null    float64
 11  1967            214 non-null    float64
 12  1968            214 non-null    float64
 13  1969            214 non-null    float64
 14  1970            214 non-null    float64
 15  1971            214 non-null    float64
 16  1972            214 non-null    float64
 17  1973            214 non-null    flo

In [22]:
World_Population.head()

,Country,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,Unnamed: 61
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,...,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,6203299.0,...,21759420.0,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2947314.0,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,83861.0,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281,NaN


In [23]:
HDI = pd.read_csv('/Users/lasayin/Desktop/Kiva_Crowdfunding/AdditionalData/Human Development Index (HDI).csv', encoding = "ISO-8859-1")

In [24]:
HDI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 62 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   HDI Rank     204 non-null    object 
 1   Country      206 non-null    object 
 2   1990         204 non-null    object 
 3   Unnamed: 3   0 non-null      float64
 4   1991         206 non-null    object 
 5   Unnamed: 5   0 non-null      float64
 6   1992         206 non-null    object 
 7   Unnamed: 7   0 non-null      float64
 8   1993         206 non-null    object 
 9   Unnamed: 9   0 non-null      float64
 10  1994         206 non-null    object 
 11  Unnamed: 11  0 non-null      float64
 12  1995         206 non-null    object 
 13  Unnamed: 13  0 non-null      float64
 14  1996         206 non-null    object 
 15  Unnamed: 15  0 non-null      float64
 16  1997         206 non-null    object 
 17  Unnamed: 17  0 non-null      float64
 18  1998         206 non-null    object 
 19  Unnamed:

In [25]:
HDI.head()

,HDI Rank,Country,1990,Unnamed: 3,1991,Unnamed: 5,1992,Unnamed: 7,1993,Unnamed: 9,...,2015,Unnamed: 53,2016,Unnamed: 55,2017,Unnamed: 57,2018,Unnamed: 59,2019,Unnamed: 61
0,169,Afghanistan,0.302,NaN,0.307,NaN,0.316,NaN,0.312,NaN,...,0.500,NaN,0.502,NaN,0.506,NaN,0.509,NaN,0.511,NaN
1,69,Albania,0.650,NaN,0.631,NaN,0.615,NaN,0.618,NaN,...,0.788,NaN,0.788,NaN,0.790,NaN,0.792,NaN,0.795,NaN
2,91,Algeria,0.572,NaN,0.576,NaN,0.582,NaN,0.586,NaN,...,0.740,NaN,0.743,NaN,0.745,NaN,0.746,NaN,0.748,NaN
3,36,Andorra,..,NaN,..,NaN,..,NaN,..,NaN,...,0.862,NaN,0.866,NaN,0.863,NaN,0.867,NaN,0.868,NaN
4,148,Angola,..,NaN,..,NaN,..,NaN,..,NaN,...,0.572,NaN,0.578,NaN,0.582,NaN,0.582,NaN,0.581,NaN


In [26]:
HDI = HDI.loc[:,~HDI.columns.str.match('Unnamed')]

In [27]:
HDI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 32 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   HDI Rank  204 non-null    object
 1   Country   206 non-null    object
 2   1990      204 non-null    object
 3   1991      206 non-null    object
 4   1992      206 non-null    object
 5   1993      206 non-null    object
 6   1994      206 non-null    object
 7   1995      206 non-null    object
 8   1996      206 non-null    object
 9   1997      206 non-null    object
 10  1998      206 non-null    object
 11  1999      206 non-null    object
 12  2000      206 non-null    object
 13  2001      206 non-null    object
 14  2002      206 non-null    object
 15  2003      206 non-null    object
 16  2004      206 non-null    object
 17  2005      206 non-null    object
 18  2006      206 non-null    object
 19  2007      206 non-null    object
 20  2008      206 non-null    object
 21  2009      206 no

In [28]:
HDI.head()

,HDI Rank,Country,1990,1991,1992,1993,1994,1995,1996,1997,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,169,Afghanistan,0.302,0.307,0.316,0.312,0.307,0.331,0.335,0.339,...,0.472,0.477,0.489,0.496,0.500,0.500,0.502,0.506,0.509,0.511
1,69,Albania,0.650,0.631,0.615,0.618,0.624,0.637,0.646,0.645,...,0.745,0.764,0.775,0.782,0.787,0.788,0.788,0.790,0.792,0.795
2,91,Algeria,0.572,0.576,0.582,0.586,0.590,0.595,0.602,0.611,...,0.721,0.728,0.728,0.729,0.736,0.740,0.743,0.745,0.746,0.748
3,36,Andorra,..,..,..,..,..,..,..,..,...,0.837,0.836,0.858,0.856,0.863,0.862,0.866,0.863,0.867,0.868
4,148,Angola,..,..,..,..,..,..,..,..,...,0.517,0.533,0.544,0.555,0.565,0.572,0.578,0.582,0.582,0.581


In [29]:
IHDI = pd.read_csv('/Users/lasayin/Desktop/Kiva_Crowdfunding/AdditionalData/Inequality-adjusted HDI (IHDI).csv', encoding = "ISO-8859-1")

In [30]:
IHDI = IHDI.loc[:, ~IHDI.columns.str.match('Unnamed')]

In [31]:
IHDI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   HDI Rank  204 non-null    object
 1   Country   206 non-null    object
 2   2010      204 non-null    object
 3   2011      206 non-null    object
 4   2012      206 non-null    object
 5   2013      206 non-null    object
 6   2014      206 non-null    object
 7   2015      206 non-null    object
 8   2016      206 non-null    object
 9   2017      206 non-null    object
 10  2018      206 non-null    object
 11  2019      206 non-null    object
dtypes: object(12)
memory usage: 19.5+ KB


In [32]:
IHDI.head()

,HDI Rank,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,169,Afghanistan,..,..,..,0.340,0.343,0.341,0.353,0.356,..,..
1,69,Albania,0.650,0.658,0.668,0.677,0.681,0.682,0.691,0.711,0.705,0.708
2,91,Algeria,..,..,..,..,..,..,..,0.591,0.594,0.596
3,148,Angola,0.311,..,0.305,0.311,0.356,0.361,0.391,0.394,0.397,0.397
4,46,Argentina,0.666,0.671,0.671,0.703,0.711,0.708,0.717,0.723,0.724,0.729
